In [2]:
import pandas as pd

In [3]:
audio = pd.read_csv('data/hot_100_audio_features.csv')
audio.head()


,SongID,Performer,Song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,"AdictoTainy, Anuel AA & Ozuna","Tainy, Anuel AA & Ozuna",Adicto,['pop reggaeton'],3jbT1Y5MoPwEIpZndDDwVq,NaN,Adicto (with Anuel AA & Ozuna),0.0,270740.0,91.0,...,10.0,-4.803,0.0,0.0735,0.01700,0.000016,0.1790,0.623,80.002,4.0
1,The Ones That Didn't Make It Back HomeJustin M...,Justin Moore,The Ones That Didn't Make It Back Home,"['arkansas country', 'contemporary country', '...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ShallowLady Gaga & Bradley Cooper,Lady Gaga & Bradley Cooper,Shallow,"['dance pop', 'pop']",2VxeLyX666F8uXCJ0dZF8B,NaN,A Star Is Born Soundtrack,0.0,215733.0,88.0,...,7.0,-6.362,1.0,0.0308,0.37100,0.000000,0.2310,0.323,95.799,4.0
3,EnemiesPost Malone Featuring DaBaby,Post Malone Featuring DaBaby,Enemies,"['dfw rap', 'melodic rap', 'rap']",0Xek5rqai2jcOWCYWJfVCF,NaN,Hollywood's Bleeding,1.0,196760.0,86.0,...,6.0,-4.169,1.0,0.2100,0.05880,0.000000,0.0955,0.667,76.388,4.0
4,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo","Yella Beezy, Gucci Mane & Quavo",Bacc At It Again,"['dfw rap', 'rap', 'southern hip hop', 'trap']",2biNa12dMbHJrHVFRt8JyO,https://p.scdn.co/mp3-preview/fa6fa6f6f363be29...,Bacc At It Again,1.0,228185.0,61.0,...,8.0,-5.725,0.0,0.1680,0.00124,0.000001,0.0716,0.856,135.979,4.0


In [4]:
# billboard = pd.read_csv('data/billboard_hot_100_1958_2019.csv')
# billboard.head()

In [6]:
genre = pd.read_csv('data/songGenre.csv')[['track_id','supergenre']].rename(columns = {'songid':'SongID'})
genre = genre.drop_duplicates()

In [7]:
join_df = genre.merge(audio, on = 'track_id', how = 'left').dropna(how='any')
target = join_df[['supergenre']]
features = join_df[['spotify_track_duration_ms','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]
feature_names = ['spotify_track_duration_ms','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']

KeyError: 'track_id'

In [1]:
for field in feature_names:
    boxplot = join_df[['supergenre',f'{field}']].boxplot(by='supgergenre')

NameError: name 'feature_names' is not defined

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=420)

In [29]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

C:\Users\msflo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.43125533731853116

In [32]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.10894291467069785, 'loudness'),
 (0.1063882766520501, 'speechiness'),
 (0.10260504883534731, 'danceability'),
 (0.0987107479149588, 'spotify_track_duration_ms'),
 (0.09568543966536859, 'acousticness'),
 (0.09230624540560088, 'valence'),
 (0.0891458264157472, 'tempo'),
 (0.08771069619942917, 'energy'),
 (0.08298234603120135, 'liveness'),
 (0.06664301403778249, 'instrumentalness'),
 (0.05496877988393817, 'key'),
 (0.013910664287878093, 'mode')]

In [33]:
print('Parameters currently in use:\n')
print(rf.get_params())

Parameters currently in use:

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [34]:
# Create the parameter grid based on the results of random search 
feature_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
#     'max_features': [2, 3],
#     'min_samples_leaf': [3, 4, 5],
#     'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
print(feature_grid)


{'bootstrap': [True], 'max_depth': [80, 90, 100, 110], 'n_estimators': [100, 200, 300, 1000]}


In [35]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(rf, feature_grid, verbose=3, n_jobs=-1)

In [36]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 16.0min finished
C:\Users\msflo\anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=200, n_jobs=None,
                                              oob_score=False,
                                              ra

In [37]:
print(grid.best_params_)
print(grid.best_score_)
predictions = grid.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

{'bootstrap': True, 'max_depth': 80, 'n_estimators': 1000}
0.41753696227178044
                 precision    recall  f1-score   support

adult standards       0.50      0.02      0.04        50
        country       0.58      0.29      0.39       262
           folk       0.00      0.00      0.00        32
         hiphop       0.51      0.29      0.37       146
          house       0.00      0.00      0.00        51
           jazz       0.36      0.05      0.09        78
          latin       1.00      0.07      0.13        14
          metal       0.67      0.04      0.07        52
          other       0.34      0.24      0.28       481
            pop       0.43      0.53      0.48       827
            r&b       1.00      0.06      0.11        51
            rap       0.61      0.42      0.50       164
           rock       0.43      0.76      0.55       972
     show tunes       0.00      0.00      0.00         5
           soul       0.35      0.09      0.14       328

       

C:\Users\msflo\anaconda3\envs\PythonData\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# import joblib
# joblib.dump(rf, 'model_RandomForest.sav')